## 중요) 이 실습은 GPU 연결이 필요합니다.
**오른쪽 위 ▼ 화살표 클릭 --> 런타임 유형 변경 --> T4 GPU 설정**  
이후 아래 코드 실행해 주세요.

-----

<br><br>
# [실습] LangChain을 이용한 RAG 만들기

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로, 질문이 주어지면 관련 있는 문서를 찾아 프롬프트에 추가하는 방식의 어플리케이션입니다.   
RAG의 과정은 아래와 같이 진행됩니다.
1. Indexing : 문서를 받아 검색이 잘 되도록 저장합니다.
1. Processing : 입력 쿼리를 전처리하여 검색에 적절한 형태로 변환합니다<br>(여기서는 수행하지 않습니다)
1. Search(Retrieval) : 질문이 주어진 상황에서 가장 필요한 참고자료를 검색합니다.
1. Augmenting : Retrieval의 결과와 입력 프롬프트를 이용해 LLM에 전달할 프롬프트를 생성합니다.
1. Generation : LLM이 출력을 생성합니다.

이번 실습에서는 웹 페이지의 결과를 받아와, 이를 기반으로 RAG를 수행하는 프로그램을 만들어 보겠습니다.

In [ ]:
# 랭체인
!pip install langchain langchain-community langchain-google-genai langchain-chroma chromadb langchain_huggingface -q

위에 발생하는 에러는 실행과 무관합니다.

In [ ]:
# 데이터 수집/전처리
!pip install rank-bm25 kiwipiepy sentence_transformers beautifulsoup4 -q

# [중요] 설치 후, **런타임 --> 세션 다시 시작** 후 실행해 주세요!

In [ ]:
import os
os.environ['GOOGLE_API_KEY']="AIxxx"

os.environ['USER_AGENT'] = 'test'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter,
    temperature = 0.5,
    max_tokens = 2048
)

In [ ]:
# Test
response = llm.invoke("알리바바의 최신 언어 모델은 무엇입니까?")
print(response.content)

## 1. `WebBaseLoader`로 웹 페이지 받아오기

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있었는데요.
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

이번에는 웹 페이지를 로드하는 `WebBaseLoader`를 통해 뉴스 기사를 읽어보겠습니다.    
WebBaseLoader는 URL의 내용을 불러오므로, URL 리스트를 먼저 전달해야 합니다.

#### 네이버 검색 연동하기
네이버 API를 사용해, 네이버 뉴스 검색 링크를 가져옵니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)   

API 사용 인증 후, 애플리케이션 등록을 통해 ID과 Secret를 발급합니다.

In [ ]:
# 스포츠 뉴스는 형식이 달라서 지원하지 않습니다...

import requests
def get_naver_news_links(query, num_links=100):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
        'X-Naver-Client-Secret': 'BvqX8mNtHu'
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    print(query, ':', len(filtered_links), 'Example:', filtered_links[0])
    return filtered_links

filtered_links = []
for topic in ['메타', '오픈AI', 'XAI', '앤트로픽','구글','알리바바']:
    filtered_links += get_naver_news_links(topic, 100)
print('Total Articles:', len(filtered_links))
print('Total Articles(Without Duplicate):',len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

WebBaseLoader를 이용해, 링크로부터 본문을 불러옵니다.

In [ ]:
# Jupyter 분산 처리를 위한 설정
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

async def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("newsct", "newsct-body")
                # newsct, newsct-body만 추출 : 네이버 뉴스 포맷에 맞는 HTML 요소
            )
        ),
        requests_per_second = 10, # 1초에 10개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    # docs = loader.load() # 기본 코드
    docs = []

    async for doc in loader.alazy_load():
        docs.append(doc)
    return docs

docs = await get_news_documents(filtered_links)
print(len(docs))

In [ ]:
docs[2]

불필요한 내용을 전처리합니다.

In [ ]:
import re

def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',
        'PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기',
        '응원 닫기',
        '구독 구독중 구독자 0 응원수 0 ',

    ]

    def clean_text(doc):
        text = doc.page_content
        # 탭과 개행문자를 공백으로 변환
        text = text.replace('\t', ' ').replace('\n', ' ')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()

        # 여러 구분자를 한번에 처리
        split_markers = [
            '구독 해지되었습니다.',
            '구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!'
        ]


        for marker in split_markers:
            parts = text.split(marker)
            if len(parts) > 1:
                if marker == '구독 해지되었습니다.':
                    text = parts[1]  # 뒷부분 사용
                else:
                    text = parts[0]  # 앞부분 사용


        # 노이즈 텍스트 제거
        for noise in noise_texts:
            text = text.replace(noise, '')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()

        doc.page_content = text

        return doc

    preprocessed_docs = []
    for doc in docs:
        # 텍스트 정제
        doc= clean_text(doc)
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


## 2. Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub


# 청크 사이즈는 RAG 성능에 매우 중요한 역할을 수행합니다!

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=600)
# 0~3000, 2400~5400, 4800~7800, ...
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

구성된 청크를 벡터 데이터베이스에 로드합니다.   
`Chroma.from_documents`는 documents의 임베딩을 구하고 이를 DB에 저장합니다.

In [ ]:
from langchain_chroma import Chroma

벡터 데이터베이스에 데이터를 저장하기 위해, 임베딩 모델을 선정합니다.   
OpenAI의 `text-embedding-3-large`나, Google의 Gemini Embedding은 빠른 속도로 연산이 가능하나, 유료 모델입니다.   
(Gemini Embedding은 일 사용량 100회로 매우 부족합니다.)

이에 따라, 오프라인 사용이 가능한 허깅페이스에서 공개 임베딩 모델을 사용하여 구현해 보겠습니다.


#### 오픈 임베딩 모델 사용하기   
- intfloat/multilingual-e5-small (500MB)   
Multilingual-E5 시리즈는 마이크로소프트의 다국어 공개 임베딩 모델입니다.

- BAAI/bge-m3 (2GB)
BGE-M3 시리즈는 BAAI의 임베딩 모델로, 현재 가장 인기가 많은 모델입니다.

- nlpai-lab/KURE-v1 (2GB)    
KURE 임베딩은 고려대학교 NLP 연구실에서 만든 모델로, BGE-M3를 한국어 텍스트로 파인 튜닝한 모델입니다.

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'nlpai-lab/KURE-v1'
# CPU 설정으로 모델 불러오기

emb_model = SentenceTransformer(model_name, device='cpu')
# 코랩 이외의 환경에서 불러오는 경우, 위 코드에 token='' 으로 HuggingFace Read 권한 토큰을 추가해야 할 수 있음

# 로컬 폴더에 모델 저장하기
emb_model.save('./embedding')

del emb_model

import gc
gc.collect()

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 허깅페이스 포맷의 임베딩 모델 불러오기
embeddings = HuggingFaceEmbeddings(model_name= './embedding',
                                   model_kwargs={'device':'cuda'}) # gpu 사용하기

In [ ]:
Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

db = Chroma.from_documents(documents=chunks,
                           embedding=embeddings,
                           persist_directory="./chroma_web",
                           collection_name='web', # DB 구분 이름

                           collection_metadata={'hnsw:space':'l2'}
                           # l2 메트릭 설정(기본값)
                           # cosine, mmr
                           )

retriever는 query에 맞춰 db에서 문서를 검색합니다.

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [ ]:
retriever.invoke("도메인 특화 언어 모델")

#### 한국어 키워드 기반 검색 추가하기

임베딩 기반의 시맨틱 검색에 추가로 키워드 검색을 연동해 보겠습니다.   
랭체인의 기본 라이브러리는 키워드 기반의 `BM25Retriever`를 지원하나, 한국어 처리를 위해서는 추가 설정이 필요합니다.

In [ ]:
# Kiwi 형태소 분석기
from kiwipiepy import Kiwi

kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]


In [ ]:
from langchain_community.retrievers import BM25Retriever, EnsembleRetriever

# 키위 형태소 분석기로 청크를 분리한 뒤, 키워드 집합 추출
# 해당 키워드 집합으로 인덱싱
bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func = kiwi_tokenize)
bm25_retriever.k = 5

retriever = db.as_retriever(search_kwargs={"k": 5})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [ ]:
ensemble_retriever

## 3. Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print()

## 4. Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
체인의 입력은 Question만 들어가므로, Context를 동시에 prompt에 넣기 위해서는 아래의 구성이 필요합니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return " \n\n---\n\n ".join(['URL: '+ doc.metadata['source'] + '\n기사 내용: ' +doc.page_content for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결


rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(format_docs(ensemble_retriever.invoke("")))

In [ ]:
rag_chain.invoke("XAI의 언어 모델 그록에 대해 설명해 주세요.")

In [ ]:
rag_chain.invoke("OpenAI의 최근 기술 발전 성과는? 관련 링크도 보여주세요")

In [ ]:
rag_chain.invoke("알리바바의 언어 모델 이름은?")

만약 Context가 포함된 RAG 결과를 보고 싶다면, RunnableParallel을 사용하면 됩니다.

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [ ]:
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

response = rag_chain_with_source.invoke("XAI의 언어 모델 그록에 대해 설명해 주세요.")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에

response

In [ ]:
print(response['context'])
print('--------')
print('Question:', response['question'])
print('Answer:', response['answer'])